
## PARTE 0: CONFIGURAÇÃO DO AMBIENTE
 

In [1]:
# 1. Instalação de bibliotecas usando o comando mágico %pip
# Garante que a instalação ocorra no kernel correto do notebook.
%pip install -q pandas numpy pyarrow fastparquet matplotlib seaborn

# 2. Importação das bibliotecas
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns

# 3. Configuração de visualização do Pandas
# Garante que todas as colunas de um DataFrame sejam exibidas
pd.set_option('display.max_columns', None)

# Configura o estilo dos gráficos para um visual mais agradável
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

print("Ambiente configurado e bibliotecas importadas com sucesso!")
print(f"Versão do Pandas: {pd.__version__}")

Note: you may need to restart the kernel to use updated packages.
Ambiente configurado e bibliotecas importadas com sucesso!
Versão do Pandas: 2.3.3



## PARTE 1: CARGA E ESTRUTURAÇÃO DO SINASC


In [2]:
# --- 1.1 Carregando os dados do SINASC (Nascidos Vivos) ---
print("Iniciando a carga dos dados do SINASC...")

# ETAPA 1: Encontrar todos os caminhos dos arquivos .parquet
# O padrão 'sinasc_2020_2022/**/*.parquet' busca recursivamente em todas as subpastas.
parquet_path = 'sinasc_2020_2022/**/*.parquet'
list_parqutet_files = glob.glob(parquet_path, recursive=True)
print(f"Encontrados {len(list_parqutet_files)} arquivos Parquet.")
# Vamos exibir os 3 primeiros caminhos para verificar se estão corretos.
print("Exemplo de caminhos encontrados:")
print(list_parqutet_files[:3])

# ETAPA 2: Iterar, ler cada arquivo, extrair metadados e armazenar em uma lista
list_dfs = []

for file_path in list_parqutet_files:
    temp_df = pd.read_parquet(file_path)   # Lê o arquivo Parquet em um DataFrame temporário
    path_parts = file_path.split(os.sep) # Divide o caminho em partes
    dir_name = path_parts[-2]  # Nome do diretório (ano)
    year_collect = int(dir_name[-4:]) # Extrai o ano dos últimos 4 caracteres
    uf_collect = dir_name[2:4]   # Extrai a UF dos caracteres na posição 2 e 3

    temp_df['ANO_COLETA'] = year_collect  # Adiciona a coluna do ano de coleta
    temp_df['UF_COLETA'] = uf_collect    # Adiciona a coluna da UF de coleta

    list_dfs.append(temp_df)  # Adiciona o DataFrame temporário à lista

# ETAPA 3: Concatenar todos os DataFrames em um único DataFrame
if list_dfs:
    df_sinasc = pd.concat(list_dfs, ignore_index=True)
    print("\nDados do SINASC carregados e consolidados com sucesso!")
    print(f"Total de registros no DataFrame final: {len(df_sinasc)}")
    print(f"Total de colunas: {len(df_sinasc.columns)}")

    display(df_sinasc.head())
else:
    print("\nErro: Nenhum DataFrame foi carregado. Verifique os arquivos Parquet.")

Iniciando a carga dos dados do SINASC...
Encontrados 984 arquivos Parquet.
Exemplo de caminhos encontrados:
['sinasc_2020_2022/DNSP2022/part-00001-bd6c59fe-baea-40ee-a750-b5ab8e63d89a-c000.gz.parquet', 'sinasc_2020_2022/DNSP2022/part-00003-bd6c59fe-baea-40ee-a750-b5ab8e63d89a-c000.gz.parquet', 'sinasc_2020_2022/DNSP2022/part-00010-bd6c59fe-baea-40ee-a750-b5ab8e63d89a-c000.gz.parquet']

Dados do SINASC carregados e consolidados com sucesso!
Total de registros no DataFrame final: 3999785
Total de colunas: 21


,CODMUNNASC,LOCNASC,CODMUNRES,DTNASC,SEXO,RACACOR,PESO,IDADEMAE,ESTCIVMAE,ESCMAE,GRAVIDEZ,CONSULTAS,RACACORMAE,DTNASCMAE,GESTACAO,SEMAGESTAC,CONSPRENAT,PARTO,CODESTAB,ANO_COLETA,UF_COLETA
0,355240,1,355240,30032022,2,4,2600,33,1,4,1,3,4,19021989,4,36,05,2,2083981,2022,SP
1,355030,1,355030,29062022,2,4,3800,36,2,3,1,4,4,03061986,5,41,16,1,7711980,2022,SP
2,355710,1,355710,21062022,2,1,3035,36,2,4,1,4,1,22021986,5,38,10,2,2081377,2022,SP
3,355210,1,355210,29042022,2,1,3420,34,1,5,1,2,1,18041988,5,38,03,2,2079704,2022,SP
4,355370,1,355370,27062022,1,4,2880,22,1,4,1,4,4,02101999,5,38,11,2,2078295,2022,SP


## ESTRATÉGIA DE OTIMIZAÇÃO: REDUÇÃO DO USO DE MEMÓRIA

In [3]:
# Função para reduzir o uso de memória dos DataFrames.

def optimize_memory_usage(df, print_log=True):
    """
    Itera sobre todas as colunas de um DataFrame e modifica o tipo de dado
    para o formato mais eficiente em memória.

    Args:
    df (pd.DataFrame): O DataFrame a ser otimizado.
    print_log (bool): Se True, imprime o log da redução de memória.

    Returns:
    pd.DataFrame: O DataFrame otimizado.
    """
    if print_log:
        # Calcula o uso de memória inicial
        mem_usage_before = df.memory_usage(deep=True).sum() / 1024**2
        print(f"Uso de memória inicial: {mem_usage_before:.2f} MB")
    
    for col in df.columns:
        col_type = df[col].dtype

        # Verifica se a coluna é numérica (inteiro ou float)
        if pd.api.types.is_numeric_dtype(col_type) and not pd.api.types.is_datetime64_any_dtype(df[col]):
            c_min = df[col].min()
            c_max = df[col].max()
            
            # Se for do tipo inteiro, tenta diminuir para o menor inteiro possível
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            # Se for do tipo float, tenta diminuir para um float menor
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        
        # Se a coluna for de texto (object)
        elif col_type == 'object':
            # Se a proporção de valores únicos for baixa, converte para 'category'
            # O tipo 'category' é muito mais eficiente para strings repetidas (ex: 'UF_COLETA')
            if df[col].nunique() / len(df[col]) < 0.5:
                df[col] = df[col].astype('category')
    
    if print_log:
        # Calcula o uso de memória final
        mem_usage_after = df.memory_usage(deep=True).sum() / 1024**2
        reduction = 100 * (mem_usage_before - mem_usage_after) / mem_usage_before
        print(f"Uso de memória final: {mem_usage_after:.2f} MB ({reduction:.2f}% de redução)")
        
    return df

print("Função 'optimize_memory_usage' definida com sucesso.")

Função 'optimize_memory_usage' definida com sucesso.


In [4]:
# Aplicando a otimização no df_sinasc

print("Otimizando df_sinasc...")
# Chamamos a função e sobrescrevemos a variável original com a versão otimizada
df_sinasc = optimize_memory_usage(df_sinasc)

# Faremos o mesmo para o df_ibge_cities quando o carregarmos.
# Por enquanto, garantimos que o df_sinasc está leve.

Otimizando df_sinasc...
Uso de memória inicial: 4605.44 MB
Uso de memória final: 199.19 MB (95.67% de redução)



## PARTE 1.2: CARREGANDO DADOS DO IBGE E ENRIQUECENDO O DF_SINASC

In [5]:
# --- 1.2.1 Carregando e Otimizando os dados do IBGE (.map() strategy)

print("Carregando e otimizando a base de cidades e estados do IBGE")
url_ibge_cities = 'https://raw.githubusercontent.com/leogermani/estados-e-municipios-ibge/master/municipios.csv'
df_ibge_cities = pd.read_csv(url_ibge_cities)
df_ibge_cities.rename(columns={
    'COD UF': 'COD_UF', 
    'COD': 'COD_MUN', 
    'NOME': 'NOME_MUN'
}, inplace=True)
df_ibge_cities = optimize_memory_usage(df_ibge_cities, print_log=False)
url_ibge_states = 'https://raw.githubusercontent.com/leogermani/estados-e-municipios-ibge/master/estados.csv'
df_ibge_states = pd.read_csv(url_ibge_states)
df_ibge_states.rename(columns={
    'COD': 'COD_UF', 
    'NOME': 'NOME_UF', 
    'SIGLA': 'SIGLA_UF'
}, inplace=True)

# --- 1.2.2 Preparando os "Dicionários de Tradução"

print("\nCriando dicionários de tradução para cidades e estados")
map_uf_code_to_name = pd.Series(df_ibge_states.NOME_UF.values, index=df_ibge_states.COD_UF).to_dict()
map_uf_code_to_sigla = pd.Series(df_ibge_states.SIGLA_UF.values, index=df_ibge_states.COD_UF).to_dict()
index_mun_6_digits = df_ibge_cities['COD_MUN'].astype(str).str[:6].astype(int) # capture apenas os 6 primeiros dígitos de COD_MUN
map_mun_code_to_name = pd.Series(df_ibge_cities.NOME_MUN.values, index=index_mun_6_digits).to_dict()
print("Dicionários criados.")

# --- 1.2.3 Preparando o df_sinasc e Aplicando o .map()
print("\nLimpando chaves do df_sinasc e aplicando .map() para enriquecer os dados")
codmunnasc_cleaned_str = df_sinasc['CODMUNNASC'].astype(str).str.extract('(\\d+)')[0].fillna('0') # Extrai apenas os dígitos
codmunnasc_limpo_int = pd.to_numeric( codmunnasc_cleaned_str, errors='coerce').astype('Int64') # Converte para Int64, permitindo NaN
cod_uf_nasc = pd.to_numeric(codmunnasc_cleaned_str.str[:2], errors='coerce').astype('Int64') # Extrai os 2 primeiros dígitos para COD_UF

#Aplicando o mapeamento.
df_sinasc['COD_UF_NASC'] = cod_uf_nasc
df_sinasc['NOME_UF_NASC'] = df_sinasc['COD_UF_NASC'].map(map_uf_code_to_name)
df_sinasc['SIGLA_UF_NASC'] = df_sinasc['COD_UF_NASC'].map(map_uf_code_to_sigla)
df_sinasc['NOME_MUNIC_NASC'] = codmunnasc_limpo_int.map(map_mun_code_to_name)
#Atualizando a coluna CODMUNNASC com a versão limpa
df_sinasc['CODMUNNASC'] = codmunnasc_limpo_int

df_sinasc_enriched = df_sinasc
print("Enriquecimento e limpeza da CODMUNNASC concluído.")
display(df_sinasc_enriched.sample(10))

#liberando memória
del df_ibge_cities
del df_ibge_states
import gc
gc.collect()
print("\nDataFrames auxiliares liberados da memória")

Carregando e otimizando a base de cidades e estados do IBGE

Criando dicionários de tradução para cidades e estados
Dicionários criados.

Limpando chaves do df_sinasc e aplicando .map() para enriquecer os dados
Enriquecimento e limpeza da CODMUNNASC concluído.


,CODMUNNASC,LOCNASC,CODMUNRES,DTNASC,SEXO,RACACOR,PESO,IDADEMAE,ESTCIVMAE,ESCMAE,GRAVIDEZ,CONSULTAS,RACACORMAE,DTNASCMAE,GESTACAO,SEMAGESTAC,CONSPRENAT,PARTO,CODESTAB,ANO_COLETA,UF_COLETA,COD_UF_NASC,NOME_UF_NASC,SIGLA_UF_NASC,NOME_MUNIC_NASC
2107752,355500,1,355500,03022021,1,4,3490,18,1,4,1,4,4,25062002,5,41,12,2,2080664,2021,SP,35,São Paulo,SP,Tupã
3927611,520870,1,520870,03102022,2,4,2325,24,1,4,1,2,4,23071998,5,41,02,1,2338564,2022,GO,52,Goiás,GO,Goiânia
3006656,230428,1,230440,04062022,1,4,3656,31,2,,1,3,4,15091990,5,40,06,2,2611295,2022,CE,23,Ceará,CE,Eusébio
2285704,280480,4,280070,19092020,1,2,3445,16,1,4,9,9,2,28102003,,,99,1,,2020,SE,28,Sergipe,SE,Nossa Senhora do Socorro
3396210,150140,1,150140,25102022,2,4,2430,29,2,5,! 1- +,4,4,21041993,5,38,08,1,2332671,2022,PA,15,Pará,PA,Belém
2130658,350410,1,350410,17052021,2,1,3190,18,1,4,1,4,1,20092002,5,40,09,2,5366828,2021,SP,35,São Paulo,SP,Atibaia
1743833,310930,1,310930,26072022,1,4,3450,17,1,4,1,4,4,10022005,5,38,08,1,2221543,2022,MG,31,Minas Gerais,MG,Buritis
2423440,354340,1,354340,28022020,1,2,2080,20,1,3,2,4,2,02111999,4,35,13,2,2082187,2020,SP,35,São Paulo,SP,Ribeirão Preto
2197582,354390,1,354390,20022021,1,1,3002,29,4,4,1,4,1,27091991,5,38,10,2,2081032,2021,SP,35,São Paulo,SP,Rio Claro
3933078,520870,1,520005,30032022,1,1,3092,25,2,4,1,4,1,16021997,5,38,08,2,2506858,2022,GO,52,Goiás,GO,Goiânia



DataFrames auxiliares liberados da memória


## DIAGNÓSTICO: Análise das colunas DTNASC e DTNASCMAE antes da limpeza.


In [6]:
print("--- INICIANDO DIAGNÓSTICO DAS COLUNAS DE DATA ---")

df_diag = df_sinasc_enriched.copy()

# 1 - VALIDAÇÃO DO ANO DE COLETA
print("\n--- Validando o Ano de Coleta ---")
valid_years = [2020, 2021, 2022]
find_years = df_diag['ANO_COLETA'].unique()
invalid_years = [year for year in find_years if year not in valid_years]

if invalid_years:
    print(f"ALERTA: A coluna 'ANO_COLETA' contém anos inválidos: {invalid_years}")
    # Definimos nosso teto de realidade manualmente
    ceiling_year = 2022
else:
    print(f"OK: Todos os anos de coleta encontrados ({list(find_years)}) são válidos.")
    # Usamos o máximo dos dados, pois confiamos neles
    ceiling_year = df_diag['ANO_COLETA'].max()
    
print(f"==> Usando {ceiling_year} como o ano máximo de referência para a análise.")

date_cols_to_diagnose = ['DTNASC', 'DTNASCMAE']

for col in date_cols_to_diagnose:
    print(f"\n\n---Análise da coluna: {col}---")
    # 2 - Analise de tipos e nulos
    print(f"Tipo de dado original: { df_diag[col].dtype}")
    null_count = df_diag[col].isnull().sum()
    print(f"Valores nulos (originais): { null_count} ({ null_count / len(df_diag):.2%})") # Exibe a contagem e a porcentagem de valores nulos

    # 3 - Analise do formato (após converter para string)
    print("\nAnálise de formato (Tamanho esperado da string 8 'DDMMAAAA')")
    series_str = df_diag[col].astype(str)
    length_counts = series_str.str.len().value_counts().sort_index()
    print(f"Contagem de tamanhos de string: { length_counts}")
    
    # Verificando conteúdo não-numérico
    no_digit = series_str[~series_str.str.isdigit().fillna(False)] # Filtra valores que não são dígitos
    if not no_digit.empty:
        print(f"\nEncontrados {len(no_digit)} valores não numericos (estão 'sujos').")
        print("Exemplos de valores não numéricos")
        display(no_digit.value_counts().head())
    else:
        print("\n Todos os valores (não nulos) são digitos. (estão 'limpos').")

    # 4 - Tentativa de conversão e análise de erros
    series_dt = pd.to_datetime( series_str, format='%d%m%Y', errors='coerce') # Converte para datetime, valores inválidos viram NaT
    count_convert_fails = series_dt.isnull().sum() # Conta quantos valores falharam na conversão
    print(f"\n Após conversão para data (ddmmyyyy):")
    print(f"Total falhas na conversão (NaT): { count_convert_fails } ({ count_convert_fails / len(df_diag):.2%})") #

    # 5 - Análise de intervalos (datas convertidas e plausíveis)
    
    valid_dates = series_dt.dropna()
    if not valid_dates.empty:
        print(f"\n Análise de intervalos das datas válidas:")
        print(f"Data mínima: { valid_dates.min().date() }")
        print(f"Data máxima: { valid_dates.max().date() }")

        #verificando datas futuras (dentro do contexto do DataFrame 2020-2022)
        future_dates = valid_dates[valid_dates.dt.year > ceiling_year]

        if not future_dates.empty:
            print(f"\n ALERTA: Encontradas { len(future_dates) } datas posteriores ao último ano válido de coleta ({ceiling_year})!")
            print("Exemplos de datas futuras:")
            print(future_dates.value_counts().head())
        
        #verificando datas muito antigas (dentro do contexto do DataFrame 2020-2022)
        old_dates = valid_dates[valid_dates.dt.year < 1920]
        if not old_dates.empty:
            print(f"\n ALERTA: Encontradas { len(old_dates) } datas anteriores a 1920!")
            print("Exemplos de datas muito antigas:")
            print(old_dates.value_counts().head())
            
print("\n--- FIM DO DIAGNÓSTICO DAS COLUNAS DE DATA ---")

--- INICIANDO DIAGNÓSTICO DAS COLUNAS DE DATA ---

--- Validando o Ano de Coleta ---
OK: Todos os anos de coleta encontrados ([np.int16(2022), np.int16(2021), np.int16(2020)]) são válidos.
==> Usando 2022 como o ano máximo de referência para a análise.


---Análise da coluna: DTNASC---
Tipo de dado original: category
Valores nulos (originais): 0 (0.00%)

Análise de formato (Tamanho esperado da string 8 'DDMMAAAA')
Contagem de tamanhos de string: DTNASC
8     3952899
9       10553
10      12163
11      13873
12       5121
13       3517
14       1659
Name: count, dtype: int64

Encontrados 46886 valores não numericos (estão 'sujos').
Exemplos de valores não numéricos


DTNASC
#25012022    9
+28112022    7
!04022022    7
 09062022    7
?20012022    6
Name: count, dtype: int64


 Após conversão para data (ddmmyyyy):
Total falhas na conversão (NaT): 46886 (1.17%)

 Análise de intervalos das datas válidas:
Data mínima: 2020-01-01
Data máxima: 2022-12-31


---Análise da coluna: DTNASCMAE---
Tipo de dado original: category
Valores nulos (originais): 0 (0.00%)

Análise de formato (Tamanho esperado da string 8 'DDMMAAAA')
Contagem de tamanhos de string: DTNASCMAE
8     3988315
9        2650
10       2943
11       3332
12       1274
13        876
14        395
Name: count, dtype: int64

Encontrados 51315 valores não numericos (estão 'sujos').
Exemplos de valores não numéricos


DTNASCMAE
             39845
02062000%        2
@04052000        2
!28092001        2
26032003*        2
Name: count, dtype: int64


 Após conversão para data (ddmmyyyy):
Total falhas na conversão (NaT): 51355 (1.28%)

 Análise de intervalos das datas válidas:
Data mínima: 1888-05-24
Data máxima: 2022-11-07

 ALERTA: Encontradas 2 datas anteriores a 1920!
Exemplos de datas muito antigas:
DTNASCMAE
1888-05-24    1
1889-06-13    1
Name: count, dtype: int64

--- FIM DO DIAGNÓSTICO DAS COLUNAS DE DATA ---


## PARTE 2: PIPELINE DE LIMPEZA E ENGENHARIA DE FEATURES

In [13]:
#Funções para etapas de limpeza e engenharia de features
def clean_to_numeric(series):
    """
    Converte uma série para numérico, extraindo apenas os dígitos.
    """
    return pd.to_numeric(series.astype(str).str.extract('(\\d+)', expand=False), errors='coerce')

def clean_to_datetime(series):
    """
    Converte uma série para datetime, extraindo o formato 'DDMMAAAA'.
    """
    return pd.to_datetime( series.astype(str).str.extract('(\\d{8})', expand=False), format='%d%m%Y', errors='coerce')

def clean_to_categorical_code(series):
    """
    Converte uma série para numérico, extraindo apenas o dígito único.
    """
    cleaned_series = series.astype(str).str.extract('(\\d)', expand=False)
    return pd.to_numeric(cleaned_series, errors='coerce')

def calculate_age_manual(row):
    """
    Calcula a idade da mãe no nascimento com base nas datas de nascimento.
    """
    nasc_date, mae_nasc_date = row['DTNASC'], row['DTNASCMAE']
    if pd.isna(nasc_date) or pd.isna(mae_nasc_date): return np.nan
    age = nasc_date.year - mae_nasc_date.year # Calcula a diferença de anos
    if (nasc_date.month, nasc_date.day) < (mae_nasc_date.month, mae_nasc_date.day): # Verifiqua se o aniversário já ocorreu no ano do nascimento
        age -= 1 # Subtrai 1 se o aniversário ainda não ocorreu
    return age

def clean_sinasc_dataframe(df):
    df_cleaned = df.copy()

    print("Iniciando limpeza profunda das colunas")
    numeric_cols = ['PESO', 'IDADEMAE', 'SEMAGESTAC', 'CONSPRENAT', 'CODESTAB']
    date_cols = ['DTNASC', 'DTNASCMAE']
    categorical_cols = ['LOCNASC', 'SEXO', 'RACACOR', 'ESTCIVMAE', 'ESCMAE', 
                        'GRAVIDEZ', 'CONSULTAS', 'RACACORMAE', 'GESTACAO', 'PARTO']
    
    for col in numeric_cols:
        if col in df_cleaned.columns:
            df_cleaned[col] = clean_to_numeric(df_cleaned[col])

    for col in date_cols:
        if col in df_cleaned.columns:
            df_cleaned[col] = clean_to_datetime(df_cleaned[col])

    for col in categorical_cols:
        if col in df_cleaned.columns:
            df_cleaned[col] = clean_to_categorical_code(df_cleaned[col])

    print("Limpeza inicial de tipos concluída.")
    print("\nIniciando engenharia de features")

    # IDADE_MAE_CRIADA
    age_series = df_cleaned.apply(calculate_age_manual, axis=1)
    age_series.loc[(age_series < 10) | (age_series > 80)] = np.nan
    df_cleaned['IDADE_MAE_CRIADA'] = age_series

    #PESO_LEN
    df_cleaned['PESO_LEN'] = df_cleaned['PESO'].astype(str).str.len()

    #Otimizando memória após limpeza
    print("\nOtimizando o DataFrame antes da conversão final...")
    df_optimized = optimize_memory_usage(df_cleaned, print_log=True)

    #Conversão final para Int64 (permitindo NaN)
    print("\nAjustando tipos de dados para inteiros com suporte a dados nulos (Int64)")
    for col in df_optimized.columns:
        if pd.api.types.is_float_dtype(df_optimized[col]): # Verifica se a coluna é do tipo float
            if (df_optimized[col].dropna() % 1 == 0).all(): # Verifica se todos os valores não nulos são "quase" inteiros
                df_optimized[col] = df_optimized[col].astype('Int64') # Converte para Int64

    print("\nPipeline de limpeza e engenharia de features concluído.")
    return df_optimized

#EXECUÇÃO DO PIPELINE DE LIMPEZA E ENGENHARIA DE FEATURES
df_sinasc_cleaned = clean_sinasc_dataframe(df_sinasc_enriched)
print( "\nVerificando o DataFrame depois da limpeza final:")
df_sinasc_cleaned.info(memory_usage='deep')
display(df_sinasc_cleaned.sample(10))



Iniciando limpeza profunda das colunas
Limpeza inicial de tipos concluída.

Iniciando engenharia de features

Otimizando o DataFrame antes da conversão final...
Uso de memória inicial: 1524.71 MB
Uso de memória final: 351.86 MB (76.92% de redução)

Ajustando tipos de dados para inteiros com suporte a dados nulos (Int64)

Pipeline de limpeza e engenharia de features concluído.

Verificando o DataFrame depois da limpeza final:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999785 entries, 0 to 3999784
Data columns (total 27 columns):
 #   Column            Dtype         
---  ------            -----         
 0   CODMUNNASC        Int64         
 1   LOCNASC           int8          
 2   CODMUNRES         category      
 3   DTNASC            datetime64[ns]
 4   SEXO              int8          
 5   RACACOR           Int64         
 6   PESO              Int64         
 7   IDADEMAE          Int64         
 8   ESTCIVMAE         Int64         
 9   ESCMAE            Int64         
 1

,CODMUNNASC,LOCNASC,CODMUNRES,DTNASC,SEXO,RACACOR,PESO,IDADEMAE,ESTCIVMAE,ESCMAE,GRAVIDEZ,CONSULTAS,RACACORMAE,DTNASCMAE,GESTACAO,SEMAGESTAC,CONSPRENAT,PARTO,CODESTAB,ANO_COLETA,UF_COLETA,COD_UF_NASC,NOME_UF_NASC,SIGLA_UF_NASC,NOME_MUNIC_NASC,IDADE_MAE_CRIADA,PESO_LEN
3712925,410840,1,410840,2022-09-06,2,4,2390,33,5,4,1,3,4,1988-11-10,4,35,4,2,6424341,2022,PR,41,Paraná,PR,Francisco Beltrão,33,6
668746,290320,1,290970,2021-12-23,1,4,3356,41,5,4,1,4,4,1980-07-06,5,38,12,1,3972925,2021,BA,29,Bahia,BA,Barreiras,41,6
1648418,320150,4,320517,2021-04-03,1,2,3160,20,1,4,1,3,2,2000-04-29,5,38,4,1,<NA>,2021,ES,32,Espírito Santo,ES,Colatina,20,6
1282490,251530,1,251530,2020-07-03,1,4,2850,32,5,3,1,2,4,1988-06-30,4,36,1,1,2357445,2020,PB,25,Paraíba,PB,Sapé,32,6
1174524,150140,1,150130,2021-03-09,2,1,2935,29,2,5,1,4,1,1991-09-06,5,38,12,2,5332427,2021,PA,15,Pará,PA,Belém,29,6
3733794,410480,1,410460,2022-12-29,1,1,2550,35,1,5,1,3,1,1987-07-06,5,37,6,2,2738899,2022,PR,41,Paraná,PR,Cascavel,35,6
3269394,330455,1,330455,2022-03-15,2,4,3280,36,1,4,1,4,4,1985-08-23,5,40,7,1,2270609,2022,RJ,33,Rio de Janeiro,RJ,Rio de Janeiro,36,6
2093086,352590,1,355670,2021-11-22,1,1,3420,27,1,4,1,3,1,1994-07-26,5,38,4,2,2078767,2021,SP,35,São Paulo,SP,Jundiaí,27,6
3462174,120040,1,120040,2021-12-06,2,4,3115,20,1,4,1,2,4,2001-06-03,5,39,1,1,2002078,2021,AC,12,Acre,AC,Rio Branco,20,6
654551,290390,2,290390,2021-04-13,1,4,2910,36,1,9,1,4,4,1985-03-12,5,38,8,1,7232519,2021,BA,29,Bahia,BA,Bom Jesus da Lapa,36,6


## PARTE 3: ANÁLISE DE QUALIDADE DE DADOS (QA) PÓS-LIMPEZA

In [14]:
# --- RELATÓRIO DE QUALIDADE DE DADOS (QA) ---
print("--- INICIANDO RELATÓRIO DE QUALIDADE DE DADOS (QA) ---")

# Vamos usar o DataFrame final e limpo
df_qa = df_sinasc_cleaned.copy()
total_records = len(df_qa)

# --- 1. ANÁLISE DE COMPLETUDE (VALORES NULOS) ---
print("\n--- 1. Análise de Completude (Valores Nulos) ---")
null_counts = df_qa.isnull().sum()
null_percentages = (null_counts / total_records) * 100
qa_report_nulls = pd.DataFrame({
    'Nulos': null_counts,
    'Porcentagem (%)': null_percentages
})
# Mostra apenas as colunas que têm algum valor nulo
display(qa_report_nulls[qa_report_nulls['Nulos'] > 0].sort_values(by='Nulos', ascending=False))

# --- 2. ANÁLISE DE VALIDADE DO INTERVALO (NUMÉRICOS) ---
print("\n--- 2. Análise de Intervalo (Ranges) para Colunas Numéricas Chave ---")
numeric_ranges = {
    'PESO': (500, 6000),
    'IDADEMAE': (10, 80),
    'IDADE_MAE_CRIADA': (10, 80),
    'SEMAGESTAC': (15, 50),
    'CONSPRENAT': (0, 50)
}

for col, (min_val, max_val) in numeric_ranges.items():
    if col in df_qa.columns:
        # Conta quantos valores estão FORA do intervalo esperado
        outliers = df_qa[(df_qa[col] < min_val) | (df_qa[col] > max_val)][col]
        if not outliers.empty:
            print(f"\nALERTA na coluna '{col}': Encontrados {len(outliers)} valores fora do intervalo esperado [{min_val}, {max_val}].")
            print("Amostra de outliers:")
            print(outliers.value_counts().head())
        else:
            print(f"OK: Todos os valores na coluna '{col}' estão dentro do intervalo esperado.")

# --- 3. ANÁLISE DE CONSISTÊNCIA CATEGÓRICA ---
print("\n--- 3. Análise de Consistência para Colunas Categóricas ---")
categorical_codes = {
    'SEXO': [1, 2, 0, 9], # Incluindo ignorado/indefinido
    'PARTO': [1, 2, 9],
    'LOCNASC': [1, 2, 3, 4, 9],
    'ESTCIVMAE': [1, 2, 3, 4, 5, 9],
    'RACACOR': [1, 2, 3, 4, 5]
}

for col, valid_codes in categorical_codes.items():
    if col in df_qa.columns:
        # Encontra valores únicos na coluna que NÃO ESTÃO na lista de códigos válidos
        invalid_values = df_qa[~df_qa[col].isin(valid_codes)][col].dropna().unique()
        if len(invalid_values) > 0:
            print(f"\nALERTA na coluna '{col}': Encontrados códigos inválidos: {list(invalid_values)}")
        else:
            print(f"OK: Todos os códigos na coluna '{col}' são válidos.")

# --- 4. ANÁLISE DE CONSISTÊNCIA LÓGICA ---
print("\n--- 4. Análise de Consistência Lógica entre Colunas ---")

# a) Comparar IDADEMAE e IDADE_MAE_CRIADA
# A diferença entre elas deve ser no máximo 1 (por causa do arredondamento do aniversário)
if 'IDADEMAE' in df_qa.columns and 'IDADE_MAE_CRIADA' in df_qa.columns:
    age_diff = abs(df_qa['IDADEMAE'] - df_qa['IDADE_MAE_CRIADA'])
    inconsistencias_idade = age_diff[age_diff > 1]
    if not inconsistencias_idade.empty:
        print(f"ALERTA: Encontradas {len(inconsistencias_idade)} inconsistências (> 1 ano) entre IDADEMAE e IDADE_MAE_CRIADA.")
    else:
        print("OK: Colunas IDADEMAE e IDADE_MAE_CRIADA são consistentes.")

print("\n--- DIAGNÓSTICO DE QUALIDADE CONCLUÍDO ---")

--- INICIANDO RELATÓRIO DE QUALIDADE DE DADOS (QA) ---

--- 1. Análise de Completude (Valores Nulos) ---


,Nulos,Porcentagem (%)
RACACORMAE,124154,3.104017
RACACOR,96421,2.410655
CONSPRENAT,73853,1.846424
SEMAGESTAC,45475,1.136936
GESTACAO,44742,1.118610
CODESTAB,41703,1.042631
IDADE_MAE_CRIADA,39931,0.998329
DTNASCMAE,39904,0.997654
ESTCIVMAE,23195,0.579906
ESCMAE,20926,0.523178



--- 2. Análise de Intervalo (Ranges) para Colunas Numéricas Chave ---

ALERTA na coluna 'PESO': Encontrados 5936 valores fora do intervalo esperado [500, 6000].
Amostra de outliers:
PESO
300    199
400    164
330    158
450    147
490    146
Name: count, dtype: Int64

ALERTA na coluna 'IDADEMAE': Encontrados 29 valores fora do intervalo esperado [10, 80].
Amostra de outliers:
IDADEMAE
99    27
8      2
Name: count, dtype: Int64
OK: Todos os valores na coluna 'IDADE_MAE_CRIADA' estão dentro do intervalo esperado.
OK: Todos os valores na coluna 'SEMAGESTAC' estão dentro do intervalo esperado.

ALERTA na coluna 'CONSPRENAT': Encontrados 18346 valores fora do intervalo esperado [0, 50].
Amostra de outliers:
CONSPRENAT
99    18313
60        5
70        4
65        3
72        3
Name: count, dtype: Int64

--- 3. Análise de Consistência para Colunas Categóricas ---
OK: Todos os códigos na coluna 'SEXO' são válidos.
OK: Todos os códigos na coluna 'PARTO' são válidos.

ALERTA na coluna 'LOCNAS

## PARTE 4: TRATAMENTO FINAL DE INCONSISTÊNCIAS

In [ ]:
# ==============================================================================
# PARTE 4: TRATamento DE INCONSISTÊNCIAS (v2 - VERSÃO COMPLETA)
# ==============================================================================

print("Iniciando o tratamento de inconsistências...")
df_tratado = df_sinasc_cleaned.copy()

# --- Ação para IDADEMAE e CONSPRENAT ---
print("Tratando valores 'Ignorado' (99) e outliers em IDADEMAE e CONSPRENAT...")
df_tratado.loc[df_tratado['IDADEMAE'] == 99, 'IDADEMAE'] = pd.NA
df_tratado.loc[df_tratado['IDADEMAE'] < 10, 'IDADEMAE'] = pd.NA
df_tratado.loc[df_tratado['CONSPRENAT'] == 99, 'CONSPRENAT'] = pd.NA
df_tratado.loc[df_tratado['CONSPRENAT'] > 50, 'CONSPRENAT'] = pd.NA

# --- Ação para LOCNASC ---
print("Tratando códigos inválidos em LOCNASC...")
df_tratado.loc[df_tratado['LOCNASC'] == 5, 'LOCNASC'] = pd.NA

# --- Ação para a consistência das idades ---
print("Harmonizando as colunas de idade da mãe...")
df_tratado['IDADEMAE'] = df_tratado['IDADE_MAE_CRIADA']

# --- Atribuição final ---
df_final = df_tratado.copy()

# --- INÍCIO DO BLOCO DE VERIFICAÇÃO RESTAURADO ---
print("\n--- Verificação Pós-Tratamento ---")

# Verifica se os valores '99' ainda existem em IDADEMAE
if 99 in df_final['IDADEMAE'].unique():
    print("ALERTA: O valor 99 ainda existe em IDADEMAE.")
else:
    print("OK: Valor 99 removido de IDADEMAE.")
print(f"Idade máxima da mãe agora: {df_final['IDADEMAE'].max()}")

# Verifica se os valores '99' ainda existem em CONSPRENAT
if 99 in df_final['CONSPRENAT'].unique():
    print("ALERTA: O valor 99 ainda existe em CONSPRENAT.")
else:
    print("OK: Valor 99 removido de CONSPRENAT.")
print(f"Consultas máximas agora: {df_final['CONSPRENAT'].max()}")

# Verifica se o código '5' ainda existe em LOCNASC
if 5 in df_final['LOCNASC'].unique():
    print("ALERTA: O código 5 ainda existe em LOCNASC.")
else:
    print("OK: Código 5 removido de LOCNASC.")

# Verifica a consistência final das idades
# Usamos .notna() para evitar erros com valores nulos na subtração
age_diff_final = abs(df_final['IDADEMAE'].dropna() - df_final['IDADE_MAE_CRIADA'].dropna())
inconsistencias_finais = age_diff_final[age_diff_final > 1].count()
print(f"Inconsistências de idade restantes: {inconsistencias_finais}")
if inconsistencias_finais == 0:
    print("OK: Colunas de idade da mãe são consistentes.")
# --- FIM DO BLOCO DE VERIFICAÇÃO RESTAURADO ---


# --- Limpeza de Memória ---
del df_sinasc_cleaned
del df_tratado
del df_diag 
gc.collect()

print("\nTratamento de inconsistências concluído. O DataFrame 'df_final' está pronto para a análise.")

Iniciando o tratamento de inconsistências...
Tratando valores 'Ignorado' (99) e outliers em IDADEMAE e CONSPRENAT...
Tratando códigos inválidos em LOCNASC...
Harmonizando as colunas de idade da mãe...

--- Verificação Pós-Tratamento ---
OK: Valor 99 removido de IDADEMAE.
Idade máxima da mãe agora: 65
OK: Valor 99 removido de CONSPRENAT.
Consultas máximas agora: 50
OK: Código 5 removido de LOCNASC.
Inconsistências de idade restantes: 0
OK: Colunas de idade da mãe são consistentes.

Tratamento de inconsistências concluído. O DataFrame 'df_final' está pronto para a análise.
